In [45]:
from collections import Counter
from pathlib import Path
import operator
import heapq

In [46]:
def read_text_file(filepath,word):
     with open(filepath) as f:
        a = f.readlines()
        count = Counter(word for line in a)
        return count[word]
    

In [47]:
read_text_file('plaintext_articles/Zambia.txt','Zambia')

613

In [48]:
def run_through_files(word):    
    pathlist = Path('plaintext_articles')
    Count_doc = dict()
    for path in pathlist.iterdir():
        path_in_str = str(path)
        count = read_text_file(path_in_str,word)
        Count_doc[path_in_str] = count
    #print(Count_doc)
    max_word = heapq.nlargest(100, Count_doc, key=Count_doc.get)
    #max_word = sorted(Count_doc, key = Count_doc.get, reverse = True)[:100]
    #max_word = max(Count_doc, key=Count_doc.get)
    print(max_word)

In [49]:
run_through_files('Zambia')

['plaintext_articles/Extinct_birds.txt', 'plaintext_articles/World_War_II.txt', 'plaintext_articles/Byzantine_Empire.txt', 'plaintext_articles/The_Holocaust.txt', 'plaintext_articles/Indo-Greek_Kingdom.txt', 'plaintext_articles/Leon_Trotsky.txt', 'plaintext_articles/List_of_lakes.txt', 'plaintext_articles/Japanese_grammar.txt', 'plaintext_articles/Vietnam_War.txt', 'plaintext_articles/List_of_sovereign_states.txt', 'plaintext_articles/Peru.txt', 'plaintext_articles/Guqin.txt', 'plaintext_articles/Elvis_Presley.txt', 'plaintext_articles/Condoleezza_Rice.txt', 'plaintext_articles/Sanskrit.txt', 'plaintext_articles/Argentina.txt', 'plaintext_articles/World_War_I.txt', 'plaintext_articles/Ronald_Reagan.txt', 'plaintext_articles/American_Civil_War.txt', 'plaintext_articles/Swastika.txt', 'plaintext_articles/La_Grande_Arm%C3%A9e.txt', 'plaintext_articles/The_Cantos.txt', 'plaintext_articles/Joseph_Stalin.txt', 'plaintext_articles/Muhammad.txt', 'plaintext_articles/Judaism.txt', 'plaintext_ar

In [51]:
import numpy as np

def make_p_matrix(matrix):
    row_sums = matrix.sum(axis=1)
    return matrix / row_sums[:]

def fix_dangling_node(H):
    # Finds which rows have all elements zeros
    # d is a column vector that identifies dangling nodes
    d = ~(H.any(axis=1))
    # w is a uniform row vector
    w = np.full(H.shape[1], 1/H.shape[1])
    S = H + d*w
    return S

def form_google_matrix(S, v=None, alpha=0.85):
    if v is None:
         v = np.full(S.shape[1], 1/S.shape[1])
    one_vector = np.ones(S.shape[0])
    return alpha*S + (1-alpha)*one_vector*v


def power_method2(H, v=None, alpha=0.85):
    if v is None:
         v = np.full(H.shape[1], 1/H.shape[1])
    d = ~(H.any(axis=1))
    w = np.full(H.shape[1], 1/H.shape[1])
    # start with pi = v
    pi = np.zeros(H.shape[0])
    pi_new = v
    cnt = 0
    print(H.shape)
    while not np.allclose(pi_new, pi):
        pi = pi_new
        pi_new = alpha * pi * H + alpha * (pi*d) * w + (1-alpha) * v
        cnt += 1
    #print("Power method2 went through {} iteration".format(cnt))
    return pi_new


def power_method1(H, v=None, alpha=0.85):
    S = fix_dangling_node(H)
    G = form_google_matrix(S, v=None, alpha=alpha)

    pi_new = np.full(G.shape[1], 1/G.shape[1])
    pi = np.zeros(G.shape[0])
    cnt = 0
    while not np.allclose(pi_new, pi):
        pi = pi_new
        pi_new = pi * G
        cnt += 1
    #print("Power method1 went through {} iteration".format(cnt))
    return pi_new



In [52]:
import matplotlib.pyplot as plt

def make_plots(g):
    def make_p_matrix(matrix):
        row_sums = matrix.sum(axis=1)
        return matrix / row_sums[:]
    
    def power_method2(H, v=None, alpha=0.85):
        if v is None:
             v = np.full(H.shape[1], 1/H.shape[1])
        d = ~(H.any(axis=1))
        w = np.full(H.shape[1], 1/H.shape[1])
        # start with pi = v
        pi = np.zeros(H.shape[0])
        pi_new = v
        cnt = 0
        while not np.allclose(pi_new, pi):
            pi = pi_new
            pi_new = alpha * pi * H + alpha * (pi*d) * w + (1-alpha) * v
            cnt += 1
        #print("Power method2 went through {} iteration".format(cnt))
        return pi_new
    x = []
    n = len(g)
    m = make_p_matrix(np.matrix(networkx.convert_matrix.to_numpy_matrix(g), dtype=np.float64))
    c = power_method2(m)
    for i in range(n):
        x.append(c.item(i))  # PageRank Probability
    
    #brandes = Brandes(g)
    """ brandes = nx.betweenness_centrality(g)
    x = []
    y = []
    n = len(g)
    for i in range(n):
        x.append(c.item(i))  # PageRank Probability
    for node in g:
        y.append(brandes[node]) # Betweenness
    plt.plot(y, x, "o")
    plt.xlabel('betweenness')
    plt.ylabel('probability output of PageRank')
    plt.show()"""
    
    
    
    

In [53]:
def make_graph_from_dataset(filepath):
    with open(filepath) as f:
        a = f.readlines()
        edges = []
        for line in a:
            if line.startswith("#"):
                continue
            else:
                a,b = line.strip().split()
                edges.append((a,b))
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

In [ ]:
g = make_graph_from_dataset('links.tsv')